In [2]:
import pandas as pd
import numpy as np

import random

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt
import seaborn as sns
import gc
import utils 

plt.style.use('ggplot')
%matplotlib inline

In [ ]:
0.538567255572 ap_hi
0.538588478581 gluc
0.538546917349 age_group_gender
0.538538052046 age_group_gluc
0.538446699606 gender_MAP
0.538589229798 gender_smoke
0.53859757607 gender_active
0.538577368412 MAP_error_group
0.538605556333 MAP_smoke
0.538595737992 MAP_active
0.538590281731 error_group_gluc
0.538613057047 error_group_active
0.538403616287 cholesterol_gluc
0.538513905577 gluc_smoke
0.538592834598 gluc_alco
0.538459912893 gluc_active
0.538497101023 smoke_active
0.538503250338 age_group_gender_error_group
0.538511181624 age_group_gender_gluc
0.538542876918 age_group_gender_smoke
0.538502938446 age_group_gender_alco
0.538421388154 age_group_error_group_gluc
0.538585787788 age_group_error_group_alco
0.538515840047 age_group_gluc_smoke
0.538604509019 age_group_gluc_alco
0.538473146062 age_group_gluc_active
0.538549234756 gender_MAP_error_group
0.538538765546 gender_MAP_smoke
0.538621130436 gender_MAP_alco
0.538224666658 gender_cholesterol_gluc
0.53854568799 gender_cholesterol_smoke
0.538519596999 gender_cholesterol_active
0.538583122431 gender_gluc_smoke
0.538546386493 gender_gluc_active
0.538433428412 gender_smoke_active
0.538530472903 gender_alco_active
0.538607528017 MAP_error_group_smoke
0.538608970885 MAP_error_group_alco
0.538496750986 MAP_gluc_active
0.538554965298 MAP_BMI_group_alco
0.538540542747 MAP_smoke_alco
0.538554965298 MAP_alco_height_group
0.538390958086 error_group_cholesterol_gluc
0.538458153646 error_group_smoke_active
0.538598031272 error_group_smoke_weight_group
0.538462902667 error_group_alco_active
0.538545756092 cholesterol_gluc_BMI_group
0.538458056167 cholesterol_gluc_smoke
0.538347090052 cholesterol_gluc_alco
0.538264506609 cholesterol_gluc_active
0.538545756092 cholesterol_gluc_height_group
0.538492455954 gluc_BMI_group_smoke
0.538578197781 gluc_smoke_alco
0.538438930193 gluc_smoke_active
0.538492455954 gluc_smoke_height_group
0.538537529542 gluc_alco_active
0.538537556667 smoke_alco_active

best_interactions = [
"ap_hi",
"gluc",
"age_group_gender",
"age_group_gluc",
"gender_MAP",
"gender_smoke",
"gender_active",
"MAP_error_group",
"MAP_smoke",
"MAP_active",
"error_group_gluc",
"error_group_active",
"cholesterol_gluc",
"gluc_smoke",
"gluc_alco",
"gluc_active",
"smoke_active",
"age_group_gender_error_group",
"age_group_gender_gluc",
"age_group_gender_smoke",
"age_group_gender_alco",
"age_group_error_group_gluc",
"age_group_error_group_alco",
"age_group_gluc_smoke",
"age_group_gluc_alco",
"age_group_gluc_active",
"gender_MAP_error_group",
"gender_MAP_smoke",
"gender_MAP_alco",
"gender_cholesterol_gluc",
"gender_cholesterol_smoke",
"gender_cholesterol_active",
"gender_gluc_smoke",
"gender_gluc_active",
"gender_smoke_active",
"gender_alco_active",
"MAP_error_group_smoke",
"MAP_error_group_alco",
"MAP_gluc_active",
"MAP_BMI_group_alco",
"MAP_smoke_alco",
"MAP_alco_height_group",
"error_group_cholesterol_gluc",
"error_group_smoke_active",
"error_group_smoke_weight_group",
"error_group_alco_active",
"cholesterol_gluc_BMI_group",
"cholesterol_gluc_smoke",
"cholesterol_gluc_alco",
"cholesterol_gluc_active",
"cholesterol_gluc_height_group",
"gluc_BMI_group_smoke",
"gluc_smoke_alco",
"gluc_smoke_active",
"gluc_smoke_height_group",
"gluc_alco_active",
"smoke_alco_active",
]

In [2]:
def clean_data(data):
    data['error_group'] = 0

    # weight/height correction
    idx = (data['height']<130) & (data['weight']>150)
    data.loc[idx, ["height", "weight"]] = data.loc[idx, ["weight", "height"]].values
#     data.loc[idx, 'error_group'] = 100-1
#     data.loc[data['weight']<20, "weight"] *= 10
#     data.loc[data['weight']<20, "weight"] *= 10
#     data.loc[data['weight']<25, "weight"] += 100

    # preasure correction

    data.loc[data["ap_hi"] < 0, "ap_hi"] *= -1
    data.loc[data["ap_lo"] < 0, "ap_lo"] *= -1
    
    for i in range(10):
        str_i = str(i)
        data['hi_' + str_i + 's'] = data['ap_hi'].apply(lambda x: str(x).count(str_i))
#         data[str(i)+'lo'] = data['ap_lo'].apply(lambda x: str(x).count(str(i)))
#         data[str(i)+'hilo'] = data[str(i)+'hi']+data[str(i)+'lo']
#         data=data.drop(str(i)+'lo', axis=1)
        for j in range(10):
            str_j = str_i + str(j)
            data['hi_' + str_j + 's'] = data['ap_hi'].apply(lambda x: str(x).count(str_j))
        

#     data.loc[(data['ap_lo'] < 20), 'error_group'] = 5
#     data.loc[(data['ap_hi'] < 50), 'error_group'] = 6
    # data.loc[(data['ap_lo'] > 250), 'error_group'] = 1
    # data.loc[(data['ap_lo'] > 4000), 'error_group'] = 2
    # data.loc[(data['ap_hi'] > 250), 'error_group'] = 3
#     data.loc[(data['ap_hi'] > 10000), 'error_group'] = 4

    data.loc[(data["ap_hi"] < 20) & (data["ap_hi"] > 10), "ap_hi"] *= 10
    data.loc[(data["ap_lo"] < 15) & (data["ap_lo"] > 2), "ap_lo"] *= 10

    idx = data['ap_hi'] > 10000
    data.loc[idx, 'ap_hi'] = 10 * (data.loc[idx, 'ap_hi'] // 1000)
    data.loc[data['ap_lo'] >= 10000, 'ap_lo'] //= 100

#     data.loc[data['ap_lo'].isin([1100])&(data['ap_hi']>160), 'ap_lo'] = 110
#     data.loc[data['ap_lo'].isin([1100]), 'ap_lo'] = 100
#     data.loc[(data['ap_lo']>250)&(data['ap_lo']<4000)&(data['ap_lo']%100==0), 'ap_lo'] /= 10
    
    manual_update = [
        
# id	age	gender	height	weight	ap_hi	ap_lo	cholesterol	gluc	smoke	alco	active	cardio	BMI
# 12494	16905	2	163	63.0	1	2088	1	1	1.0	0.0	1.0	0	23.711845
# 42591	18191	2	162	63.0	140	1900	1	1	1.0	0.0	1.0	1	24.005487
# 78873	20323	1	168	68.0	130	1900	1	1	0.0	0.0	1.0	0	24.092971
# 51749	18419	1	169	62.0	1	2088	1	1	0.0	0.0	1.0	-5	21.707923
        (12494, ['ap_hi', 'ap_lo'], [120, 80]),
        (42591, ['ap_hi', 'ap_lo'], [140, 90]), # ?
        (78873, ['ap_hi', 'ap_lo'], [130, 100]), # ?
        (51749, ['ap_hi', 'ap_lo'], [120, 80]),
        
# 57807	20496	1	164	62.0	70	1100	1	1	0.0	0.0	0.0	0	23.051755
# 60477	18716	1	171	80.0	1	1088	1	1	0.0	0.0	1.0	1	27.358845
# 91198	18182	2	186	95.0	100	901	2	2	0.0	0.0	1.0	0	27.459822
# 6580	19079	1	176	92.0	1	1099	1	1	0.0	NaN	1.0	-5	29.700413
        (57807, ['ap_hi', 'ap_lo'], [170, 100]),
        (60477, ['ap_hi', 'ap_lo'], [110, 80]),
        (91198, ['ap_hi', 'ap_lo'], [100, 90]),
        (6580,  ['ap_hi', 'ap_lo'], [110, 90]),
        
# 44701	22801	1	163	115.0	20	170	1	1	0.0	0.0	1.0	1	43.283526
# 94673	22551	1	169	88.0	10	160	3	3	0.0	0.0	0.0	1	30.811246
        (44701, ['ap_hi', 'ap_lo'], [120, 70]),
        (94673,  ['ap_hi', 'ap_lo'], [110, 60]),
        
    ]
    for idx, cols, update in manual_update:
        data.loc[data['id']==idx, cols] = update
        

    return data

def new_features(data):
    data["BMI"] = 10000 * data["weight"] / (data["height"] * data["height"])
    data["BMI_1"] = 100 * data["weight"] / data["height"]
    data["BMI_3"] = 1000000 * data["weight"] / (data["height"] * data["height"] * data["height"])
    data["BMI_4"] = 100000000 * data["weight"] / (data["height"] * data["height"] * data["height"] * data["height"])
    data["ap_dif"] = data["ap_hi"] - data["ap_lo"]
    data["MAP"] = (data["ap_lo"] * 2 + data["ap_dif"] )/ 3.0
    
    data["age_years"] = np.round(data["age"] / 365)
    
#     age_bins = [0, 10000, 14000, 14980, 15700, 16420, 17140, 17890, 18625, 19355, 20090, 20820, 21555, 22280, 22990, 24000]
#     age_names = [25, 30, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64]
    age_bins = [0, 14000, 14980, 15700, 16420, 17140, 17890, 18625, 19355, 20090, 20820, 21555, 22280, 22990, 24000]
    age_names = [30, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64]
    data["age_group"] = pd.cut(data['age'], age_bins, labels=age_names).astype('str')
    
    bins = [0, 70, 90, 120, 140, 160, 190, 20000]
    names = list(range(len(bins)-1))
    data["ap_hi_group"] = pd.cut(data['ap_hi'], bins, labels=names).astype('str')
    
    bins = [-1, 40, 60, 80, 90, 100, 2000000]
    names = list(range(len(bins)-1))
    data["ap_lo_group"] = pd.cut(data['ap_lo'], bins, labels=names).astype('str')
    
    data["weight_group"] = pd.qcut(data['weight'], 10, labels=False).astype('str')
    
    data["height_group"] = pd.qcut(data['height'], 10, labels=False).astype('str')
    data["BMI_group"] = pd.qcut(data['height'], 10, labels=False).astype('str')

    return data

In [3]:
train = pd.read_csv('train.csv', sep=';')
test = pd.read_csv('test.csv', sep=';', na_values='None')

train = clean_data(train)
test = clean_data(test)

train = new_features(train)
test = new_features(test)

X_train = train.drop([ 'cardio'], axis=1)
y_train = train['cardio'].values.ravel()
X_test = test.drop([], axis=1)

columns_to_interact = ['age_group', 'gender', 'MAP', 'error_group', 'cholesterol', 'gluc', 
                       'BMI_group', 'smoke', 'alco', 'active', "ap_hi_group", "ap_lo_group",
                      "weight_group", "height_group"]
temp = utils.generate_interactions(X_train, columns_to_interact, 2)
X_train = pd.concat((X_train, temp), axis=1)
X_test = pd.concat((X_test, utils.generate_interactions(X_test, columns_to_interact, 2)), axis=1)
print(temp.columns)

X = pd.concat((X_train, X_test), axis=0)

# from sklearn.preprocessing import LabelEncoder
# from collections import defaultdict
# label_encoder_dict = defaultdict(LabelEncoder)
# columns_to_encode = ['age_group'] + list(temp.columns)


# X[columns_to_encode].apply(lambda x: label_encoder_dict[x.name].fit(x))
# X_train[columns_to_encode] = X_train[columns_to_encode].apply(lambda x: label_encoder_dict[x.name].transform(x))
# X_test[columns_to_encode] = X_test[columns_to_encode].apply(lambda x: label_encoder_dict[x.name].transform(x))

Index(['age_group_gender', 'age_group_MAP', 'age_group_error_group',
       'age_group_cholesterol', 'age_group_gluc', 'age_group_BMI_group',
       'age_group_smoke', 'age_group_alco', 'age_group_active',
       'age_group_ap_hi_group', 'age_group_ap_lo_group',
       'age_group_weight_group', 'age_group_height_group', 'gender_MAP',
       'gender_error_group', 'gender_cholesterol', 'gender_gluc',
       'gender_BMI_group', 'gender_smoke', 'gender_alco', 'gender_active',
       'gender_ap_hi_group', 'gender_ap_lo_group', 'gender_weight_group',
       'gender_height_group', 'MAP_error_group', 'MAP_cholesterol', 'MAP_gluc',
       'MAP_BMI_group', 'MAP_smoke', 'MAP_alco', 'MAP_active',
       'MAP_ap_hi_group', 'MAP_ap_lo_group', 'MAP_weight_group',
       'MAP_height_group', 'error_group_cholesterol', 'error_group_gluc',
       'error_group_BMI_group', 'error_group_smoke', 'error_group_alco',
       'error_group_active', 'error_group_ap_hi_group',
       'error_group_ap_lo_group', 'err

In [4]:
from sklearn.model_selection import train_test_split
__, idx = train_test_split(list(range(X_train.shape[0])), test_size=0.101, random_state=42)
X_train.loc[idx, 'alco'] = np.nan
__, idx = train_test_split(list(range(X_train.shape[0])), test_size=0.101, random_state=43)
X_train.loc[idx, 'smoke'] = np.nan
__, idx = train_test_split(list(range(X_train.shape[0])), test_size=0.096, random_state=44)
X_train.loc[idx, 'active'] = np.nan

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

def impute_nans(model, X_train, X_test, columns, target, perform_cv = False, scoring='accuracy'):
    X = pd.concat((X_train, X_test), axis=0)
    train_idx = ~X[target].isnull()
    model = clone(model)
    model.fit(X.loc[train_idx, columns], X.loc[train_idx, target])
    X_train.loc[X_train[target].isnull(), target] = model.predict(X_train.loc[X_train[target].isnull(), columns])
    X_test.loc[X_test[target].isnull(), target] = model.predict(X_test.loc[X_test[target].isnull(), columns])
    
    if perform_cv:
        n_folds = 7
        kf = StratifiedKFold(random_state=111, n_splits=n_folds, shuffle=True)
        scores = cross_val_score(model, X.loc[train_idx, columns], X.loc[train_idx, target], cv=kf, scoring=scoring)
        print(target, np.mean(scores), 'on', n_folds, 'folds', 'with mean on train', X.loc[train_idx, target].mean())
    
params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'n_estimators': 200,
    'subsample': 0.8,
    'n_jobs': 4,
    'random_state': 2707,
    'silent': True,
}
clf = xgb.XGBClassifier(**params)

columns = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc','BMI', 'ap_dif', 'MAP']
impute_nans(clf, X_train, X_test, columns, 'smoke')#, perform_cv = True)

columns = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc','BMI', 'ap_dif', 'MAP', ]
impute_nans(clf, X_train, X_test, columns, 'alco')#, perform_cv = True)

columns = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc','BMI', 'ap_dif', 'MAP', ]
impute_nans(clf, X_train, X_test, columns, 'active')#, perform_cv = True)

In [84]:
use_columns = [
    'age_group', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active'
'BMI',
 'MAP',
]
mean_columns = ['ap_hi', 'gluc',  'MAP_active', 'gender_MAP', 'cholesterol_gluc', 'gluc_smoke',
                'gluc_active', 'gluc_alco', 'smoke_active'] #, 'age_group_MAP', 'gender_MAP'

for i in range(10):
        str_i = str(i)
        use_columns.append(str_i)
#         for j in range(10):
#             str_j = str_i + str(j)
#             use_columns.append(str_j)

In [85]:
import gc
gc.collect()

180

In [86]:
X_train['age_group'] = X_train['age_group'].astype('float')
X_test['age_group'] = X_test['age_group'].astype('float')

In [87]:
# X_train['age'].astype('float').hist()

In [ ]:
import utils
params = {
    'colsample_bytree': 0.8,
    #'gamma': 0.125,
    'learning_rate': 0.1,
#     'max_depth': 4,
    # 'min_child_weight': 1,
    'n_estimators': 224,
    'subsample': 0.8,
    # 'reg_alpha': 0.0,
    # 'reg_lambda': 2.0,
    
    'n_jobs': 4,
    'random_state': 2707,
    'silent': True,
}

utils.execute_model(xgb.XGBClassifier(**params),
              X_train,
              y_train,
              X_test,
              model_name="model1",
               mean_columns = mean_columns,
              use_columns = use_columns,
              n_splits=15,
              n_folds=10,
#               stratification_groups=rew,
              alpha=20,
#               seed=150
             )
gc.collect()
# 10 folds logloss:
# [0.53291704422372443, 0.5443312752011974, 0.54886863255657625, 0.53516634300212229, 0.53726825344604312, 0.52536659773190864, 0.54180775521269742, 0.54391921066351068, 0.53444038602695909, 0.53302140455759295]
# mean: 0.537710690262
# std: 0.00662097344978
# 15 Splits logloss:
# [0.53710825758951974, 0.5411839522050822, 0.53794312811962219, 0.53275829491839444, 0.54280702872209363, 0.53248789428678656, 0.53687309152124529, 0.53289596940278317, 0.53803566650621171, 0.53810661018600148, 0.53506605126855322, 0.54186916663160944, 0.53686088768378049, 0.5389666476164654, 0.53387849979650193]
# mean: 0.537122743097
# std: 0.0031674337146
# model1 results saved!


10 folds logloss:
[0.53231306374958842, 0.54420662366905981, 0.54842461542683196, 0.535387869636611, 0.53754438333581911, 0.52527889954493323, 0.54199811644724516, 0.54412114582696636, 0.53505923510908071, 0.53387943758984302]
mean: 0.537821339034
std: 0.00653570603012
15 Splits logloss:
[0.53766918366350369, 0.54163396015904608, 0.53825134418438514, 0.53240877384548857, 0.54294308869287422, 0.53232140182769472, 0.53674238947843411, 0.53348780648576288, 0.53813878365249357, 0.5384852398646236, 0.53545692908778231, 0.54188396179550025, 0.53703232042461779, 0.53907891625399962, 0.534198687297195]
mean: 0.537315519114
std: 0.00321031655597


In [ ]:
params = {
    'colsample_bytree': 0.8,
    #'gamma': 0.125,
    'learning_rate': 0.1,
    # 'max_depth': 9,
    # 'min_child_weight': 1,
    'n_estimators': 200,
    'subsample': 0.8,
    # 'reg_alpha': 0.0,
    # 'reg_lambda': 2.0,
    'tree_method': 'hist',
    'grow_policy': 'lossguide',
    
    'n_jobs': 1,
    'random_state': 2707,
    'silent': True,
}

utils.execute_model(xgb.XGBClassifier(**params),
              X_train,
              y_train,
              X_test,
              model_name="model2",
              mean_columns = mean_columns,
              use_columns = use_columns,
              n_splits=15,
              n_folds=10,
#               stratification_groups=rew,
              alpha=20
             )
gc.collect()
# 10 folds logloss:
# [0.53260452189206642, 0.54356527722526271, 0.54912191522116438, 0.53542797206519244, 0.53751433990565378, 0.52485793828534233, 0.54176204704486508, 0.54439543156634218, 0.53539547971672896, 0.53310668233666803]
# mean: 0.537775160526
# std: 0.00669792931425
# 15 Splits logloss:
# [0.53761403867575286, 0.54134343673111951, 0.53794831141736743, 0.53246545070220785, 0.54284968435188097, 0.53256692490579061, 0.5367211861235498, 0.5332958718095373, 0.53813802041428871, 0.53835573232952039, 0.53518534024897735, 0.54186608721225504, 0.53689329263676422, 0.53901083277677597, 0.53386733317131929]
# mean: 0.5372081029
# std: 0.00318011438105
# model2 results saved!

In [ ]:
import lightgbm as lgb
params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
#     'num_leaves': 2**3,
#     'min_child_weight': 1,
    'n_estimators': 44,
    'subsample': 0.8,
#     'reg_alpha': 1.2,
#     'reg_lambda': 0.0,
       
    'nthread': 1,
    'seed': 2707,
    'silent': True,
}

utils.execute_model(lgb.LGBMClassifier(**params),
              X_train,
              y_train,
              X_test,
              model_name="model3",
              mean_columns = mean_columns,
              use_columns = use_columns,
              n_splits=15,
              n_folds=10,
#               stratification_groups=rew,
              alpha=20
             )
gc.collect()
# 10 folds logloss:
# [0.53343111141185251, 0.54411143767050563, 0.54946981792109495, 0.53572293596647347, 0.53856368378590369, 0.52596408000293715, 0.54156698655989965, 0.54385119115682679, 0.53417427867914336, 0.53393460506511559]
# mean: 0.538079012822
# std: 0.00646690224982
# 15 Splits logloss:
# [0.53730108009319477, 0.54179601041640013, 0.53841153030613909, 0.53273513566356923, 0.54331052022759863, 0.53307633902951568, 0.53718380231723861, 0.53375225691727657, 0.53848255338605422, 0.53859474874074931, 0.53585761204767779, 0.54196359812544492, 0.53744967867465532, 0.53940724907473092, 0.53469619495508147]
# mean: 0.537601220665
# std: 0.00311597365397
# model3 results saved!

In [ ]:
models = ['model1','model2','model3']
result = utils.merge_models(models, method='mean')
pd.DataFrame(result).to_csv('models1234_mean.csv', index=False, header=False, sep=';')

In [ ]:
# 0.537735274001	0.537762262313 = 0.5439345
# 0.538040557134	0.538014744278 = 0.5436639
# 0.5374	0.5374 = 0.5440495
# 0.537603843701	0.537606129043 = 0.5448082


In [67]:
from sklearn.ensemble import RandomForestClassifier

In [68]:
clf = RandomForestClassifier(n_estimators=200)
utils.execute_model(clf,
              X_train,
              y_train,
              X_test,
              model_name="model4",
              mean_columns = mean_columns,
              use_columns = use_columns,
              n_splits=15,
              n_folds=10,
#               stratification_groups=rew,
              alpha=10
             )
gc.collect()


10 folds logloss:
[0.5543050725828268, 0.56359446718627748, 0.57278779500887278, 0.56983658349332789, 0.56201723791499913, 0.55028675369097724, 0.56385165763553347, 0.56825805402233198, 0.55726434523944335, 0.5560393571236395]
mean: 0.56182413239
std: 0.00690638475108
15 Splits logloss:
[0.5605010133863253, 0.56559901189539297, 0.56241674779000828, 0.55666357077310402, 0.56705411306086928, 0.55564568960945637, 0.5600505197044281, 0.5581296510135515, 0.56106971348481849, 0.56321321064169305, 0.55821054194957775, 0.56487479349010461, 0.56269827452627041, 0.56253516355072253, 0.56016618504413995]
mean: 0.561255213328
std: 0.00315521153011
model4 results saved!


1647